In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import math
import random
from typing import List, Dict, Tuple, Any
from utils.helper import load_multiple_data
from utils.config import get_secret

In [6]:
def create_positive_pairs_for_contrastive_learning(
    data: List[Dict[str, str]],
    include_identity_pairs: bool = False
) -> List[Tuple[str, str]]:
    """
    주어진 데이터 형식에서 Contrastive Learning을 위한 긍정 쌍(Positive Pair)을 생성합니다.

    각 의미 그룹 내 4가지 버전의 문장들 사이에서 가능한 모든 순서 있는 쌍을 생성하여,
    모든 버전이 서로를 긍정적인 예시로 끌어당기도록 학습 데이터를 구성합니다.
    (예: MultipleNegativesRankingLoss와 함께 사용하기 적합)

    Args:
        data: 각 요소가 {'EtoK': ..., 'KtoE': ..., 'English': ..., 'Korean': ...}
              형식의 딕셔너리인 리스트. 각 딕셔너리는 같은 의미를 가진 문장 그룹임.
        include_identity_pairs: 동일한 문장으로 구성된 쌍 (예: ('text_a', 'text_a'))을
                                 포함할지 여부. 일반적으로 Contrastive Loss에서는 False.

    Returns:
        긍정적인 (anchor, positive) 문장 쌍의 리스트.
        예: [('EtoK_sent1', 'KtoE_sent1'), ('EtoK_sent1', 'English_sent1'), ...]
    """
    training_pairs: List[Tuple[str, str]] = []
    sentence_keys: List[str] = ['EtoK', 'KtoE', 'English', 'Korean']

    print(f"원본 데이터 그룹 수: {len(data)}")

    for item in data:
        # 현재 의미 그룹의 4가지 버전 문장 추출
        sentences_in_group: List[str] = [item[key] for key in sentence_keys if key in item and item[key]]

        # 유효한 문장이 2개 미만인 경우 건너뛰기 (쌍을 만들 수 없음)
        if len(sentences_in_group) < 2:
            print(f"Warning: 유효한 문장이 2개 미만인 그룹 발견. 건너<0xEB><0x8B>니다: {item}")
            continue

        # 그룹 내에서 모든 순서 있는 쌍 생성
        for i in range(len(sentences_in_group)):
            for j in range(len(sentences_in_group)):
                if i == j and not include_identity_pairs:
                    continue # 자기 자신과의 쌍은 제외 (옵션)

                anchor = sentences_in_group[i]
                positive = sentences_in_group[j]

                # 문장이 비어있지 않은 경우에만 쌍 추가
                if anchor and positive:
                    training_pairs.append((anchor, positive))

    print(f"생성된 긍정 쌍(Positive Pair) 수: {len(training_pairs)}")
    if len(data) > 0:
         print(f"예상 쌍 수 (그룹당 최대 {len(sentence_keys)*(len(sentence_keys)-1) if not include_identity_pairs else len(sentence_keys)**2}): 약 {len(data) * len(sentence_keys)*(len(sentence_keys)-1 if not include_identity_pairs else len(sentence_keys))}")

    return training_pairs

In [9]:

# 1. 모델 로드
model_name = "hyoseok1989/bge-m3-finetuned"
model = SentenceTransformer(model_name)

# 2. 데이터 준비 (이전 단계에서 생성한 함수 사용)
data_dir = "dataset/"
if isinstance(data_dir, str):
    data = load_multiple_data(data_dir)
else:
    data = load_multiple_data(data_dir)
# positive_pairs: List[Tuple[str, str]]
positive_pairs = create_positive_pairs_for_contrastive_learning(data)
train_examples = [InputExample(texts=[pair[0], pair[1]]) for pair in positive_pairs]

You try to use a model that was created with version 4.1.0, however, your version is 3.4.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





데이터 디렉토리: dataset/
로드할 파일: dataset/code-switch.json
로드된 데이터 일부 (첫 번째 항목): {'EtoK': 'Could you explain how 양자 컴퓨팅 and 양자 얽힘 make it possible to solve certain problems faster than classical methods?', 'KtoE': '고전적 방식보다 더 효율적인 quantum computing과 quantum entanglement의 핵심 원리는 무엇인가요?', 'English': 'Could you explain how quantum computing and quantum entanglement make it possible to solve certain problems faster than classical methods?', 'Korean': '양자 컴퓨팅과 양자 얽힘이 일부 문제를 고전적 방법보다 더 빠르게 해결하도록 만드는 핵심 원리는 무엇인가요?'}
로드할 파일: dataset/code-switch1.json
로드된 데이터 일부 (첫 번째 항목): {'topic': 'science/technology', 'type': 'fact', 'complexity': 'simple', 'contents': 'brief', 'EtoK': 'The 원자 is the basic unit of matter.', 'KtoE': '원자는 matter의 기본 단위이다.', 'English': 'The atom is the basic unit of matter.', 'Korean': '원자는 물질의 기본 단위이다.'}
로드할 파일: dataset/code-switch2.json
로드된 데이터 일부 (첫 번째 항목): {'topic': 'science/technology', 'type': 'fact', 'complexity': 'simple', 'contents': 'brief', 'EtoK': 'The 블랙홀 is a space regio

In [33]:
# 3. DataLoader 준비
batch_size = 16 # GPU 메모리에 맞게 조정
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# 4. 손실 함수 정의 (라이브러리 내장 함수 사용)
# MultipleNegativesRankingLoss는 InputExample(texts=[anchor, positive]) 형태를 기대하며,
# 배치 내 다른 모든 샘플을 자동으로 네거티브로 사용합니다.
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [36]:
# 5. 모델 Fine-tuning 실행
num_epochs = 1
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 예: 10% 웜업
output_save_path = './bge-m3-codeswitch-finetuned'

model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=output_save_path,
          show_progress_bar=True,
          checkpoint_path=output_save_path + '-checkpoints', # 체크포인트 저장 경로
          checkpoint_save_steps=len(train_dataloader) // 2 # 예: 에폭당 2번 저장
         )

print(f"Fine-tuning 완료. 모델 저장 경로: {output_save_path}")

KeyboardInterrupt: 